<a href="https://colab.research.google.com/github/daniel9511/Curso-Spark/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Instalar SDK java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# descargar spark
!wget -q https://archive.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz

# descomprimir la version de spark
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

# descargar findspark
!pip install -q findspark

# descargar pyspark
!pip install -q pyspark



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
# importar findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
# crear un data frame median la lectura de un archivo de texto

df = spark.read.text('./data/dataTXT.txt')

In [ ]:
df.show()

+--------------------+
|               value|
+--------------------+
|Estamos en el cur...|
|En este capítulo ...|
|En esta sección e...|
|y en este ejemplo...|
+--------------------+



In [ ]:
df.show(truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|Estamos en el curso de pyspark                                         |
|En este capítulo estamos estudiando el API SQL de Saprk                |
|En esta sección estamos creado dataframes a partir de fuentes de datos,|
|y en este ejemplo creamos un dataframe a partir de un texto plano      |
+-----------------------------------------------------------------------+



In [ ]:
# crear un dataframe mediante la elctura de un archivo csv
df_csv = spark.read.csv('./data/dataCSV.csv')
df_csv.show()
#df_csv.printSchema()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|        _c0|          _c1|                 _c2|                 _c3|        _c4|                 _c5|                 _c6|    _c7|   _c8|     _c9|         _c10|                _c11|             _c12|            _c13|                _c14|                _c15|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|vid

In [ ]:
df_csv = spark.read.option('header','true').csv('./data/dataCSV.csv')
df_csv.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [ ]:
# leer arcivo de texto con un delimitador diferente

df_txt = spark.read.option('delimiter','|').option('header', 'true').csv('./data/dataTab.txt')
df_txt.show()

+----+----+----------+-----+
|pais|edad|     fecha|color|
+----+----+----------+-----+
|  MX|  23|2021-02-21| rojo|
|  CA|  56|2021-06-10| azul|
|  US|  32|2020-06-02|verde|
+----+----+----------+-----+



In [ ]:
# cread un dataframe a partir de un json porporcionan un schema

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

json_schema = StructType (
    [StructField('color', StringType(), True),
     StructField('edad', IntegerType(), True),
     StructField('fecha', DateType(), True),
     StructField('pais', StringType(), True)
     ]
)

df_json = spark.read.schema(json_schema).json('./data/dataJSON.json')

df_json.show()

+-----+----+----------+----+
|color|edad|     fecha|pais|
+-----+----+----------+----+
| rojo|null|2021-02-21|  MX|
| azul|null|2021-06-10|  CA|
|verde|null|2020-06-02|  US|
+-----+----+----------+----+



In [ ]:
# crear un dataframe a aprtir de un archivo parquet

df_parquet = spark.read.parquet('./data/dataPARQUET.parquet')
df_parquet.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

**TRASNFORMACIONES**

In [ ]:
df = spark.read.parquet('./data/dataPARQUET.parquet')
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
# Seleccionar valores distintos de la columna video_id
df.select("video_id").distinct().where((df.video_id == 'B5HORANmzHw') & (df.trending_date == '17.14.11')).show()


+-----------+
|   video_id|
+-----------+
|B5HORANmzHw|
+-----------+



In [ ]:
df.selectExpr("COUNT(video_id)").show()
df.selectExpr("COUNT(DISTINCT(video_id))").show()
df.selectExpr("video_id").where(df.video_id == 'B5HORANmzHw').show()
print(df.select("video_id").distinct().count()) # se necesita variable
df.groupBy("video_id").count().show()
df.select("video_id").where(df.video_id == 'B5HORANmzHw').show()

+---------------+
|count(video_id)|
+---------------+
|          48137|
+---------------+

+------------------------+
|count(DISTINCT video_id)|
+------------------------+
|                    6837|
+------------------------+

+-----------+
|   video_id|
+-----------+
|B5HORANmzHw|
|B5HORANmzHw|
+-----------+

6837
+--------------------+-----+
|            video_id|count|
+--------------------+-----+
|         bAkEd8r7Nnw|    8|
|         eijd-yjXY9E|    2|
|         npcqBt_e4k0|    4|
|         LeWtF5y9-6Q|    5|
|         GhcqN2FDAnA|    4|
|         v_CMMWCN5nQ|    7|
|         R8WBN3fJmwM|    6|
|         oKuPJ7zF0_k|    3|
|         B3JFSL8AA70|    5|
|         f6Egj7ncOi8|    7|
|         8gE6cek7F30|    7|
|         EdkK29-TWJk|    4|
|         8szK9FBpdPI|    1|
|         6gFj1XJ6b5o|    3|
|\nhttp://www.Mast...|    1|
|         wOFuVNiAJQQ|    1|
|         PpElRBQ-yGc|    7|
|         q11UD-6XT-8|    5|
|         IzQwbRdh5Ts|    3|
|         IfdihPR__WI|    4|
+---------------

In [ ]:
df_duplicates = df.dropDuplicates(["video_id"])
print(df_duplicates.count())

6837


In [ ]:
df.orderBy(["video_id", "trending_date"], ascending=False).show()

+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|zzQsGL_F9_c|     18.06.02|Budweiser | Beer ...|    Budweiser|         24|2018-01-30T18:30:...|"Budweiser"|"Supe...| 199041|  1415|     121|           62|https://i.ytimg.c...|            False|           False|                 False|Th

In [ ]:
from pyspark.sql.functions import length

df_with_length = df.withColumn("length", length(df.video_id))

df_with_length.show()

from pyspark.sql.functions import expr, col
df = df.withColumn('likes', col('likes').cast('int')).withColumn('dislikes', col('dislikes').cast('int'))
df.printSchema()
df.withColumn("DIFF LIKES", col("likes") - col("dislikes")).show()



+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|length|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|           

In [ ]:
# Renombrar la columna 'video_id' a 'vid_id'
df_renamed = df.withColumnRenamed("video_id", "vid_id")

# Mostrar el resultado
df_renamed.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|     vid_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [ ]:
# drop

df.drop("video_id").show()

+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           False|                 False|SHANTELL'S CHANNE...|


In [ ]:
# sample

df.sample(0.5).show()

+-----------+-------------+--------------------+-------------------+-----------+--------------------+--------------------+--------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|      channel_title|category_id|        publish_time|                tags|   views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+-------------------+-----------+--------------------+--------------------+--------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|1ZAPwfrtAFY|     17.14.11|The Trump Preside...|    LastWeekTonight|         24|2017-11-13T07:30:...|"last week tonigh...| 2418783| 97185|    6146|        12703|https://i.ytimg.c...|            False|           Fal

In [ ]:
# randomsplit

print(df.randomSplit([0.5, 0.5])) # dsitribucion para df entrenamiento y df test

[DataFrame[video_id: string, trending_date: string, title: string, channel_title: string, category_id: string, publish_time: string, tags: string, views: string, likes: int, dislikes: int, comment_count: string, thumbnail_link: string, comments_disabled: string, ratings_disabled: string, video_error_or_removed: string, description: string], DataFrame[video_id: string, trending_date: string, title: string, channel_title: string, category_id: string, publish_time: string, tags: string, views: string, likes: int, dislikes: int, comment_count: string, thumbnail_link: string, comments_disabled: string, ratings_disabled: string, video_error_or_removed: string, description: string]]


In [ ]:
# manejar vacias

df.na.drop(subset=['video_id']).show()

df.fillna(0, subset=['likes']).show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [7]:
# agregaciones

df = spark.read.parquet('./data/vuelos.parquet')
df.show()
df.printSchema()


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [15]:
# count, countdistinc,

df = spark.read.parquet('./data/dataframe.parquet')
df.show()
df.printSchema()
from pyspark.sql.functions import count
df.select(count("cantidad")).show()
df.selectExpr("count(cantidad)").show()
df.na.drop(subset=['nombre']).show()

from pyspark.sql.functions import countDistinct

df.select(countDistinct("color")).show()

+------+-----+--------+
|nombre|color|cantidad|
+------+-----+--------+
|  Jose| azul|    1900|
|  null| null|    1700|
|  null| rojo|    1300|
|  Juan| rojo|    1500|
+------+-----+--------+

root
 |-- nombre: string (nullable = true)
 |-- color: string (nullable = true)
 |-- cantidad: long (nullable = true)

+---------------+
|count(cantidad)|
+---------------+
|              4|
+---------------+

+---------------+
|count(cantidad)|
+---------------+
|              4|
+---------------+

+------+-----+--------+
|nombre|color|cantidad|
+------+-----+--------+
|  Jose| azul|    1900|
|  Juan| rojo|    1500|
+------+-----+--------+

+---------------------+
|count(DISTINCT color)|
+---------------------+
|                    2|
+---------------------+



In [31]:
from pyspark.sql.functions import sum, sumDistinct, avg

df = spark.read.parquet('./data/vuelos.parquet')

df.show()
df.printSchema()

df.select(sum('DISTANCE')).show()
df.select(sumDistinct('DISTANCE')).show()
df.select(avg('DISTANCE')).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

/content/spark-3.2.3-bin-hadoop3.2/python/pyspark/sql/functions.py:215: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----------------------+
|sum(DISTINCT DISTANCE)|
+----------------------+
|               1442300|
+----------------------+

+-----------------+
|    avg(DISTANCE)|
+-----------------+
|822.3564947305235|
+-----------------+



In [32]:
from pyspark.sql.functions import desc
df.show()
(df.groupBy('ORIGIN_AIRPORT').count().orderBy(desc('count'))).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [33]:
df.groupBy('ORIGIN_AIRPORT').agg({'DISTANCE': 'avg'}).show()

+--------------+------------------+
|ORIGIN_AIRPORT|     avg(DISTANCE)|
+--------------+------------------+
|           BGM|             378.0|
|           PSE|1373.7316421895862|
|           INL| 157.7787456445993|
|           DLG|             329.0|
|         12888|             113.0|
|           MSY| 735.6827388928976|
|           PPG|            2599.0|
|         12003| 600.8157894736842|
|         15041|             222.0|
|           GEG| 603.3416096791162|
|           SNA|   797.80853524081|
|           BUR| 454.5088146540314|
|           GRB|254.78303626306084|
|           GTF| 592.1627670396745|
|         14986| 714.3803278688524|
|         13851| 607.0505789152955|
|           IDA| 303.0952380952381|
|         11150|              83.0|
|         15412| 518.5182829888712|
|           GRR| 476.5688335638543|
+--------------+------------------+
only showing top 20 rows



In [ ]:
# join

from pyspark.sql.functions import col

join_df = empleados.join(departamentos, empleados.departamento_id == departamentos.id, 'left')
join_df.show()

In [52]:
from pyspark.sql.functions import col, desc

df_players = spark.read.option('header', True).csv('./data/players.csv')
df_players.groupBy('country_of_citizenship').count().orderBy(desc('count')).show()
df_players.printSchema()

+----------------------+-----+
|country_of_citizenship|count|
+----------------------+-----+
|                 Spain| 1379|
|                 Italy| 1319|
|                France| 1311|
|                Turkey| 1300|
|               England| 1112|
|                Brazil| 1095|
|           Netherlands| 1091|
|               Ukraine| 1076|
|                Russia| 1039|
|               Germany|  950|
|                Greece|  904|
|              Portugal|  856|
|               Denmark|  838|
|               Belgium|  809|
|              Scotland|  697|
|             Argentina|  413|
|                Serbia|  286|
|               Croatia|  227|
|               Senegal|  221|
|               Nigeria|  215|
+----------------------+-----+
only showing top 20 rows

+----------------------+-----+
|country_of_citizenship|count|
+----------------------+-----+
|                 Spain| 1379|
|                 Italy| 1319|
|                France| 1311|
|                Turkey| 1300|
|            

In [65]:
df_appearances = spark.read.option('header', True).csv('./data/appearances.csv')
df_appearances.printSchema()

df_appearances_fix =  df_appearances.withColumn('red_cards', col('red_cards').cast('int'))

df_transform = df_appearances_fix.join(df_players, df_appearances_fix.player_id == df_players.player_id)

df_transform.groupBy('name').sum('red_cards').orderBy(desc('sum(red_cards)')).show()

root
 |-- player_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- appearance_id: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- player_club_id: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- minutes_played: string (nullable = true)
 |-- yellow_cards: string (nullable = true)
 |-- red_cards: string (nullable = true)

+--------------------+--------------+
|                name|sum(red_cards)|
+--------------------+--------------+
|             marcelo|             6|
|              hilton|             5|
|    domenico-berardi|             5|
|     mario-balotelli|             5|
|          thomas-lam|             5|
|             rodrigo|             5|
|        granit-xhaka|             5|
|        jordan-amavi|             4|
|       ramon-leeuwin|             4|
|          jordi-amat|             4|
|          allan-nyom|             4|
|       heung-min-son|             4|
